# Getting Started

In [1]:
%reload_ext autoreload
%autoreload 2
from recsys.MovieLens import MovieLens
from surprise import SVD

In [2]:
def BuildAntiTestSetForUser(test_subject, trainset):
    fill = trainset.global_mean

    anti_testset = []
    
    u = trainset.to_inner_uid(str(test_subject))
    
    user_items = set([j for (j, _) in trainset.ur[u]])
    anti_testset += [(trainset.to_raw_uid(u), trainset.to_raw_iid(i), fill) for
                             i in trainset.all_items() if
                             i not in user_items]
    return anti_testset

In [3]:
# Pick an arbitrary test subject
test_subject = 85

lens, data, _ = MovieLens.load()

user_ratings = lens.get_user_ratings(test_subject)
loved = []
hated = []
for ratings in user_ratings:
    if (float(ratings[1]) > 4.0):
        loved.append(ratings)
    if (float(ratings[1]) < 3.0):
        hated.append(ratings)

print("\nUser ", test_subject, " loved these movies:")
for ratings in loved:
    print(lens.get_movie_name(ratings[0]))
print("\n...and didn't like these movies:")
for ratings in hated:
    print(lens.get_movie_name(ratings[0]))

Loading movie ratings...

Computing movie popularity ranks so we can measure novelty later...

User  85  loved these movies:
Jumanji (1995)
GoldenEye (1995)
Braveheart (1995)
Jerky Boys, The (1995)
LÃ©on: The Professional (a.k.a. The Professional) (LÃ©on) (1994)
Pulp Fiction (1994)
Stargate (1994)
Shawshank Redemption, The (1994)
Star Trek: Generations (1994)
Clear and Present Danger (1994)
Speed (1994)
True Lies (1994)
Fugitive, The (1993)
Jurassic Park (1993)
Terminator 2: Judgment Day (1991)
Mission: Impossible (1996)
Rock, The (1996)

...and didn't like these movies:
Grumpier Old Men (1995)
Mortal Kombat (1995)
Postman, The (Postino, Il) (1994)
Casper (1995)
Lord of Illusions (1995)
Mighty Morphin Power Rangers: The Movie (1995)
Prophecy, The (1995)
Dolores Claiborne (1995)
Heavenly Creatures (1994)
Little Women (1994)
Miracle on 34th Street (1994)
Nell (1994)
Poison Ivy II (1996)
Tank Girl (1995)
While You Were Sleeping (1995)
Wes Craven's New Nightmare (Nightmare on Elm Street Pa

In [4]:
# Building recommendation model
train_set = data.build_full_trainset()

algo = SVD()
algo.fit(train_set)

In [5]:
# Computing recommendations
test_set = BuildAntiTestSetForUser(test_subject, train_set)
predictions = algo.test(test_set)

recommendations = []

print("Loading movie ratings...")
data = lens.load_movielens_data()

print ("\nWe recommend:")
for user_id, movie_id, true_rating, predicted_rating, _ in predictions:
    intMovieID = int(movie_id)
    recommendations.append((intMovieID, predicted_rating))

recommendations.sort(key=lambda x: x[1], reverse=True)

for ratings in recommendations[:10]:
    print(lens.get_movie_name(ratings[0]))

Loading movie ratings...

We recommend:
Princess Bride, The (1987)
Casablanca (1942)
Ocean's Eleven (2001)
Producers, The (1968)
Lord of the Rings: The Return of the King, The (2003)
Smoke (1995)
North by Northwest (1959)
Run Lola Run (Lola rennt) (1998)
Inglourious Basterds (2009)
Modern Times (1936)
